시각화 없이 데이터 뽑기 위한 파일

In [147]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 문자열 비교 및 대체 용도
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [148]:
# 데이터 로드

df = pd.read_csv('./data/netflix_user_data_unclean.csv')

origin_cnt = df.shape[0]

In [149]:
def displayCountNonNumeric(df, col_name, show_all=False):
    """
    숫자로 예상되는 컬럼의 숫자가 아닌 데이터를 노출한다.
    """
    # 1. 숫자로 변환될 수 없는 값은 NaN으로 강제 변환
    numeric_series = pd.to_numeric(df[col_name], errors='coerce')

    # 2. 숫자로 변환된 값(NaN이 아님)과 원래 NaN이 아닌 값(문자열 등)을 구분
    # True: 숫자가 아니거나, 공백이거나, 원래 NaN이었던 값
    non_numeric_mask = numeric_series.isna()

    # 3. 원본 데이터프레임에서 숫자가 아닌 행만 필터링
    non_numeric_data = df[non_numeric_mask]

    # 4. value_counts()를 사용하여 빈도수를 계산하되,
    #    NaN 값도 결과에 포함시키도록 dropna=False 설정
    df_result = non_numeric_data[col_name].value_counts(dropna=False).reset_index()

    # 컬럼 이름 변경
    df_result.columns = [col_name, 'count']
    # df_result['percent'] = df_result['count'] / df.shape[0] * 100
    df_result['percent'] = (df_result['count'] / df.shape[0] * 100).round(4)

    pd.set_option('display.float_format', '{:.4f}'.format)

    if show_all:
        # 전체 행 출력을 위한 pandas 옵션 설정
        pd.set_option('display.max_rows', None)  # 모든 행 출력
    # 5. 결과 출력
    display(df_result)

    if show_all:
        # # 출력 후 기본값으로 복원 (선택사항)
        pd.reset_option('display.max_rows')

    pd.set_option('display.float_format', '{:.2f}'.format)

def displayPatterns(df, col, show_all=False):
    """
    문자열 컬럼 데이터 분포 보기
    """
    if show_all:
        # 전체 행 출력을 위한 pandas 옵션 설정
        pd.set_option('display.max_rows', None)  # 모든 행 출력

    df_result = df[col].value_counts(dropna=False).reset_index()
    df_result['percent'] = (df_result['count'] / df.shape[0] * 100).round(4)

    pd.set_option('display.float_format', '{:.4f}'.format)

    print(df_result)
    
    pd.set_option('display.float_format', '{:.2f}'.format)

    if show_all:
        # # 출력 후 기본값으로 복원 (선택사항)
        pd.reset_option('display.max_rows')

def fuzzy_match_and_clean(wrong_value, allowed_values, THRESHOLD = 80):
    """
    주어진 오류 값(wrong_value)를 허용 목록(allowed_values)과 비교하여
    유사도가 임계값(THRESHOLD) 이상이면 가장 유사한 값으로 대체합니다.
    """
    if wrong_value in allowed_values or pd.isna(wrong_value) or wrong_value == '':
        return wrong_value
    
    # process.extractOne: 목록에서 가장 유사한 항목 1개를 찾아냄
    # 결과: ('가장 유사한 장르', 유사도 점수)
    best_match = process.extractOne(wrong_value, allowed_values)
    
    match_value = best_match[0]
    score = best_match[1]
    
    if score >= THRESHOLD:
        # 유사도 점수가 높으면 정상 장르로 대체
        return match_value
    else:
        # 유사도 점수가 낮으면(너무 다른 단어이면) np.nan 등으로 남겨서 후속 처리를 유도
        return wrong_value
    
def printMiddle(before_cnt, after_cnt):
    print('-'*77)
    print("최초", origin_cnt, "처리 전", before_cnt, "처리 후", after_cnt, '제거됨', before_cnt - after_cnt, '누적제거', origin_cnt - after_cnt)

In [150]:
# 1. 전체 중복 데이터 처리

before_cnt = df.shape[0]
df = df.drop_duplicates().reset_index(drop=True) # drop=True 중복 데이터 남김
after_cnt = df.shape[0]

printMiddle(before_cnt, after_cnt)

-----------------------------------------------------------------------------
최초 119779 처리 전 119779 처리 후 118334 제거됨 1445 누적제거 1445


In [151]:
# 2. Churn status (클래스 데이터 이므로 별도 처리)
#    - 결측치 처리
#    - 문자형 오류 데이터 치환 처리
#    - 극 소수 데이터 처리
#      - Maybe - 0.35%

before_cnt = df.shape[0]

allowed_list = [
'Yes',
'No',
'Maybe',
]
col_name = 'Churn status'
# cleaned_col_name = f'Cleaned {col_name}'
cleaned_col_name = col_name

print('처리전')

displayPatterns(df, cleaned_col_name, True)

# 유사 값 변경처리
df[cleaned_col_name] = df[col_name].apply(lambda x: fuzzy_match_and_clean(x, allowed_list, 50))


# Churn status는 필수이기 때문에 무조건 드롭한다.
df = df.dropna(subset=[col_name])

# 데이터 수가 작음으로 일단 제거
index_to_drop = df[df[col_name] == 'Maybe'].index
df.drop(index_to_drop, axis=0, inplace=True)

print('처리후')

displayPatterns(df, cleaned_col_name, True)

after_cnt = df.shape[0]

printMiddle(before_cnt, after_cnt)

처리전
   Churn status   count  percent
0           Yes  104771  88.5384
1            No   12403  10.4813
2         Maybe     423   0.3575
3           NaN     388   0.3279
4            Ny      33   0.0279
5            zo      33   0.0279
6            yo      32   0.0270
7            Nx      31   0.0262
8            xo      28   0.0237
9            Nz      27   0.0228
10          Yey      26   0.0220
11          yes      24   0.0203
12          Yxs      22   0.0186
13          zes      20   0.0169
14          Yex      19   0.0161
15          xes      17   0.0144
16          Yzs      14   0.0118
17          Yez      13   0.0110
18          Yys      10   0.0085
처리후
  Churn status   count  percent
0          Yes  104936  89.2898
1           No   12587  10.7102
-----------------------------------------------------------------------------
최초 119779 처리 전 118334 처리 후 117523 제거됨 811 누적제거 2256


In [152]:
# 3. Customer ID - 결측치 처리가 의미가 있는지 확인 필요
#    - 결측치 401건 처리

# col_name = 'Customer ID'

# # 결측치 공백처리, NaN은 빈 문자열로 대체 (str 메서드의 오류 방지)
# df[col_name] = df[col_name].replace(np.nan, '', regex=True)

# # 정상 코드 패턴 정의 (C + 6자리 숫자)
# NORMAL_PATTERN = r'^C\d{6}$'

# # 정상 패턴에 일치하지 않는 행(오류 데이터)을 필터링하고 공백으로 대체
# # str.match()의 결과를 Boolean Series로 얻고, ~ (틸드) 연산자로 True/False를 반전시킴
# # 즉, 'is_normal'이 False인 모든 행을 선택하여 ''로 바꿉니다.
# df.loc[~df[col_name].str.match(NORMAL_PATTERN), col_name] = np.nan

In [153]:
# ----------------------------------------------------------------------
# 회의 후 다시 할것 -일단 결측치 있으면 중위값으로 대체
# ----------------------------------------------------------------------
# 4. 숫자형 컬럼 숫자 아닌 오류데이터 제거
#    - Subscription Length (Months)
#    - Customer Satisfaction Score (1-10)
#    - Daily Watch Time (Hours)
#    - Engagement Rate (1-10)
#    - Support Queries Logged
#    - Age
#    - Monthly Income ($)
#    - Promotional Offers Used
#    - Number of Profiles Created


target_cols = [
    'Subscription Length (Months)',
    'Customer Satisfaction Score (1-10)',
    'Daily Watch Time (Hours)',
    'Engagement Rate (1-10)',
    'Support Queries Logged',
    'Age',
    'Monthly Income ($)',
    'Promotional Offers Used',
    'Number of Profiles Created'
]

before_cnt = df.shape[0]

for col_name in target_cols:
    # 값을 숫자 혹은 NaN으로 변경처리
    df[col_name] = pd.to_numeric(df[col_name], errors='coerce')
    # 컬럼 유형을 float로 변경처리
    df[col_name] = df[col_name].astype(float)

    # # 일단 결측치 있으면 일단 드롭
    # df = df.dropna(subset=[col_name])

    # 일단 결측치 있으면 일단 중위값 대체
    median_value = df[col_name].median()
    df[col_name].fillna(median_value, inplace=True)

after_cnt = df.shape[0]
printMiddle(before_cnt, after_cnt)

-----------------------------------------------------------------------------
최초 119779 처리 전 117523 처리 후 117523 제거됨 0 누적제거 2256


In [154]:
# ----------------------------------------------------------------------
# 회의 후 다시 할것 - 일단 이상치 날림
# ----------------------------------------------------------------------
# 5. 범위가 정해진 숫자형 컬럼 데이터 정리 필요 1~10으로 한정
#    - Engagement Rate (1-10)
#    - Customer Satisfaction Score (1-10)

before_cnt = df.shape[0]

for col_name in ['Engagement Rate (1-10)', 'Customer Satisfaction Score (1-10)']:
    # 1보다 작으면 날림
    index_to_drop = df[df[col_name] < 1.0].index
    df.drop(index_to_drop, axis=0, inplace=True)
    # 10보다 크면 날림
    index_to_drop = df[10.0 < df[col_name]].index
    df.drop(index_to_drop, axis=0, inplace=True)


after_cnt = df.shape[0]
printMiddle(before_cnt, after_cnt)

-----------------------------------------------------------------------------
최초 119779 처리 전 117523 처리 후 116790 제거됨 733 누적제거 2989


In [155]:
# 6. 범주화 필요
#    - Daily Watch Time (Hours)
#      - 시청시간 시간단위로 반올림
#    - Age
#      - 연령 10단위로 반올림

# 시청시간 시간단위로 반올림
df['Daily Watch Time (Hours)'] = df['Daily Watch Time (Hours)'].round(0)

# 연령 10단위로 반올림
df['Age'] = df['Age'].round(-1)




In [156]:
# ----------------------------------------------------------------------
# 회의 후 다시 할것 -일단 결측치 있으면 일단 드롭
# ----------------------------------------------------------------------
# 7. 문자형 오류 데이터 유사 문자 치환 처리
#    - Device Used Most Often
#      - 'Laptop', 'Mobile', 'Tablet', 'Smart TV', 'Desktop'
#    - Genre Preference
#      - 'Sci-Fi', 'Romance', 'Drama', 'Thriller', 'Documentary', 'Action', 'Comedy', 'Dramedy', 
#    - Region
#      - 'Asia', 'Africa', 'Europe', 'South America', 'North America', 'Eurasia', 
#    - Payment History
#      - 'Delayed', 'On-Time', 'Late', 
#    - Subscription Plan
#      - 'Standard', 'Premium', 'Basic',


target_list = [
    ('Device Used Most Often', ['Laptop', 'Mobile', 'Tablet', 'Smart TV', 'Desktop'], 80)
    ,('Genre Preference', ['Sci-Fi', 'Romance', 'Drama', 'Thriller', 'Documentary', 'Action', 'Comedy', 'Dramedy'], 80)
    ,('Region', ['Asia', 'Africa', 'Europe', 'South America', 'North America', 'Eurasia'], 70)
    ,('Payment History (On-Time/Delayed)', ['Delayed', 'On-Time', 'Late'], 80)
    ,('Subscription Plan', ['Standard', 'Premium', 'Basic'], 80)
]


before_cnt = df.shape[0]

for i, (col_name, allowed_list, percent) in enumerate(target_list):
    cleaned_col_name = col_name
    df[cleaned_col_name] = df[col_name].apply(lambda x: fuzzy_match_and_clean(x, allowed_list, percent))    
    df = df.dropna(subset=[col_name])


after_cnt = df.shape[0]
printMiddle(before_cnt, after_cnt)

-----------------------------------------------------------------------------
최초 119779 처리 전 116790 처리 후 115020 제거됨 1770 누적제거 4759


In [158]:
# 8. 극 소수 데이터 처리
#    - Device Used Most Often
#      - Smart_Television - 0.32%
#    - Genre Preference
#      - Dramedy - 0.31%
#    - Region
#      - Eurasia - 0.32%
#    - Payment History
#      - Late - 0.32%

before_cnt = df.shape[0]

# 대체 처리
col_name = 'Device Used Most Often'
df[col_name] = df[col_name].replace('Smart_Television', 'Smart TV')

# 드롭 처리
col_name = 'Genre Preference'
index_to_drop = df[df[col_name] == 'Dramedy'].index
df.drop(index_to_drop, axis=0, inplace=True)

# 드롭 처리
col_name = 'Region'
index_to_drop = df[df[col_name] == 'Eurasia'].index
df.drop(index_to_drop, axis=0, inplace=True)

# 드롭 처리
col_name = 'Payment History (On-Time/Delayed)'
index_to_drop = df[df[col_name] == 'Late'].index
df.drop(index_to_drop, axis=0, inplace=True)

after_cnt = df.shape[0]
printMiddle(before_cnt, after_cnt)


-----------------------------------------------------------------------------
최초 119779 처리 전 115020 처리 후 114113 제거됨 907 누적제거 5666


In [159]:
df = df.reset_index(drop=True)

In [160]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114113 entries, 0 to 114112
Data columns (total 16 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Customer ID                         113856 non-null  object 
 1   Subscription Length (Months)        114113 non-null  float64
 2   Customer Satisfaction Score (1-10)  114113 non-null  float64
 3   Daily Watch Time (Hours)            114113 non-null  float64
 4   Engagement Rate (1-10)              114113 non-null  float64
 5   Device Used Most Often              114113 non-null  object 
 6   Genre Preference                    114113 non-null  object 
 7   Region                              114113 non-null  object 
 8   Payment History (On-Time/Delayed)   114113 non-null  object 
 9   Subscription Plan                   114113 non-null  object 
 10  Churn status                        114113 non-null  object 
 11  Support Queries Logged    

In [161]:
df.to_csv(f'./data/netflix_user_data_clean_kbs.csv', index=False)